<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Init-data" data-toc-modified-id="Init-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Init data</a></span><ul class="toc-item"><li><span><a href="#Estaciones" data-toc-modified-id="Estaciones-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Estaciones</a></span></li><li><span><a href="#Data-Mediciones" data-toc-modified-id="Data-Mediciones-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data Mediciones</a></span></li></ul></li><li><span><a href="#Georreferenciar-estaciones" data-toc-modified-id="Georreferenciar-estaciones-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Georreferenciar estaciones</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from zopilote.solr import swipe_radius_cvegeo
from zopilote.make_influence_areas import compute_influence_area

%matplotlib inline

Package `lxml` missing. Pretty print will be disabled


# Init data

## Estaciones

In [2]:
estaciones = pd.read_csv('/data/calidad_aire_presidencia/data_estaciones_sinaica.csv')
mask = (estaciones.lat.between(14, 34.5)) & (estaciones.long.between(-120, -70))
estaciones = estaciones[mask].rename(columns={'id':'estacionesid'})

print(estaciones.shape)
estaciones.head()

(180, 8)


,_id,lat,long,estacionesid,nombre,codigo,redesid,date-insert
0,5b8850e3e2705c1932e82ff8,21.873311,-102.320803,31,CBTIS,CBT,30,2018-08-30 20:17:39.218
1,5b8850e3e2705c1932e82ff9,21.846392,-102.288431,32,Secretaría de Medio Ambiente,SMA,30,2018-08-30 20:17:39.218
2,5b8850e3e2705c1932e82ffa,21.883781,-102.295825,33,Centro,CEN,30,2018-08-30 20:17:39.218
3,5b8850e3e2705c1932e82ffb,31.859917,-116.593722,36,Secundaria,SPABC20,31,2018-08-30 20:17:39.218
4,5b8850e3e2705c1932e82ffc,32.631317,-115.444631,38,UABC,SPABC12,32,2018-08-30 20:17:39.218


## Data Mediciones

In [3]:
mediciones = pd.read_csv('/data/calidad_aire_presidencia/data_mediciones_todas_estaciones_2.csv')
print(mediciones.shape)
mediciones.head()

(3240000, 8)


,city,estacionesid,fecha,hora,parametro,state,validoorig,valororig
0,Durango,58,2018-08-01,13,PM10,Durango,1,988.61
1,Durango,58,2018-08-01,14,PM10,Durango,1,988.61
2,Durango,58,2018-08-01,15,PM10,Durango,1,988.60
3,Durango,58,2018-08-01,16,PM10,Durango,1,988.63
4,Durango,58,2018-08-01,17,PM10,Durango,1,988.63


In [4]:
estaciones_ciudad = mediciones[['city', 'state', 'estacionesid']].drop_duplicates()
estaciones_ciudad = estaciones_ciudad.merge(estaciones[['estacionesid', 'lat', 'long', 'nombre']],
                                            on='estacionesid')
print(estaciones_ciudad.shape)
estaciones_ciudad.head()

(121, 6)


,city,state,estacionesid,lat,long,nombre
0,Durango,Durango,58,24.034472,-104.645028,ITD
1,Durango,Durango,59,24.051556,-104.611361,IPN
2,Durango,Durango,60,24.026778,-104.692975,SRNyMA
3,Gómez Palacio,Durango,65,25.551222,-103.502528,Campestre
4,Celaya,Guanajuato,68,20.517500,-100.815556,Policía


In [5]:
estaciones_ciudad.groupby('city').estacionesid.nunique().sort_values(ascending=False).head()

city
Valle de México    32
Monterrey          11
Guadalajara        10
Toluca              6
León                3
Name: estacionesid, dtype: int64

# Georreferenciar estaciones

In [20]:
from tqdm import tqdm_notebook
georref = []

for i, row in tqdm_notebook(estaciones_ciudad.iterrows()):
    aux = swipe_radius_cvegeo(row.lat, row.long, row.estacionesid)
    georref.append(aux)
    
georref = pd.concat(georref)
print(georref.shape)
georref.head()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

Revisar coordenadas 160
Revisar coordenadas 241

(123, 3)


,cvegeo,index_name,radius
0,1000500013698027,58,0.001
0,1000500014573001,59,0.001
0,1000500015266011,60,0.001
0,1000700010166050,65,0.001
0,1100700010226014,68,0.001


In [25]:
estaciones_georref = georref.drop_duplicates(subset=['index_name']).rename(columns={'index_name':'estacionesid'})
estaciones_georref = estaciones_georref.merge(estaciones, on='estacionesid',
                                              how='right').set_index('estacionesid').reset_index()
print(estaciones_georref.shape)
estaciones_georref.head()

(180, 10)


,estacionesid,cvegeo,radius,_id,lat,long,nombre,codigo,redesid,date-insert
0,58,1000500013698027,0.001,5b8850e3e2705c1932e8300b,24.034472,-104.645028,ITD,ITD,42,2018-08-30 20:17:39.219
1,59,1000500014573001,0.001,5b8850e3e2705c1932e8300c,24.051556,-104.611361,IPN,IPN,42,2018-08-30 20:17:39.219
2,60,1000500015266011,0.001,5b8850e3e2705c1932e8300d,24.026778,-104.692975,SRNyMA,SRN,42,2018-08-30 20:17:39.219
3,65,1000700010166050,0.001,5b8850e3e2705c1932e8300e,25.551222,-103.502528,Campestre,CAM,43,2018-08-30 20:17:39.219
4,68,1100700010226014,0.001,5b8850e3e2705c1932e83011,20.517500,-100.815556,Policía,POL,45,2018-08-30 20:17:39.219


In [26]:
# estaciones_georref.to_csv('/data/calidad_aire_presidencia/data_estaciones_sinaica_cvegeo.csv', index=False)

In [13]:
georref = []

for i, row in estaciones_ciudad[estaciones_ciudad.nombre=='Ajusco'].iterrows():
    aux = swipe_radius_cvegeo(row.lat, row.long, row.estacionesid, swipe_max=2, swipe_step=0.01)
    georref.append(aux)
    
georref = pd.concat(georref)
print(georref.shape)
georref.head()

(1, 3)


,cvegeo,index_name,radius
0,0901200192460002,241,1.611


In [21]:
estaciones_georref = pd.read_csv('/data/calidad_aire_presidencia/data_estaciones_sinaica_cvegeo.csv')

estaciones_georref['cvegeo'] = np.where(estaciones_georref.estacionesid==241, '0901200192460002',
                                        estaciones_georref.cvegeo)
estaciones_georref['radius'] = np.where(estaciones_georref.estacionesid==241, 1.611,
                                        estaciones_georref.radius)

estaciones_georref[estaciones_georref.nombre=='Ajusco']

,estacionesid,cvegeo,radius,_id,lat,long,nombre,codigo,redesid,date-insert
105,241,0901200192460002,1.611,5b8850e3e2705c1932e8306a,19.154286,-99.162778,Ajusco,AJU,119,2018-08-30 20:17:39.225


In [22]:
estaciones_georref.to_csv('/data/calidad_aire_presidencia/data_estaciones_sinaica_cvegeo.csv', index=False)